In [1]:
import nshutils as nu
import rich

nu.pretty(log_level="WARNING")

In [2]:
from __future__ import annotations

import os
from pathlib import Path

import mattertune.configs as MC
from mattertune import MatterTuner

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


def hparams():
    hparams = MC.MatterTunerConfig.draft()

    # Model hparams
    hparams.model = MC.JMPBackboneConfig.draft()
    hparams.model.graph_computer = MC.JMPGraphComputerConfig.draft()
    hparams.model.graph_computer.pbc = True

    hparams.model.ckpt_path = Path("/mnt/shared/checkpoints/jmp-l.pt")
    hparams.model.ignore_gpu_batch_transform_error = True
    hparams.model.optimizer = MC.AdamWConfig(lr=1.0e-4)

    hparams.model.properties = []
    energy = MC.EnergyPropertyConfig(loss=MC.MAELossConfig())
    hparams.model.properties.append(energy)
    forces = MC.ForcesPropertyConfig(loss=MC.MAELossConfig(), conservative=False)
    hparams.model.properties.append(forces)
    stresses = MC.StressesPropertyConfig(loss=MC.MAELossConfig(), conservative=False)
    hparams.model.properties.append(stresses)

    # Data hparams
    hparams.data = MC.ManualSplitDataModuleConfig.draft()
    hparams.data.train = MC.OMAT24DatasetConfig.draft()
    hparams.data.train.src = Path("/mnt/datasets/salex/val/")
    hparams.data.batch_size = 1

    # Trainer hparams
    hparams.trainer.additional_trainer_kwargs = {"fast_dev_run": True}

    hparams = hparams.finalize()
    return hparams


hp = hparams()
rich.print(hp)

MatterTunerConfig(
    data=ManualSplitDataModuleConfig(
        batch_size=1,
        num_workers='auto',
        pin_memory=True,
        train=OMAT24DatasetConfig(type='omat24', src=Path('/mnt/datasets/salex/val')),
        validation=None
    ),
    model=JMPBackboneConfig(
        properties=[
            EnergyPropertyConfig(
                name='energy',
                dtype='float',
                loss=MAELossConfig(name='mae', reduction='mean'),
                loss_coefficient=1.0,
                type='energy'
            ),
            ForcesPropertyConfig(
                name='forces',
                dtype='float',
                loss=MAELossConfig(name='mae', reduction='mean'),
                loss_coefficient=1.0,
                type='forces',
                conservative=False
            ),
            StressesPropertyConfig(
                name='stresses',
                dtype='float',
                loss=MAELossConfig(name='mae', reduction='mean'),
                loss_coefficient=1.0,
                type='stresses',
                conservative=False
            )
        ],
        optimizer=AdamWConfig(
            name='AdamW',
            lr=0.0001,
            eps=1e-08,
            betas=(0.9, 0.999),
            weight_decay=0.01,
            amsgrad=False
        ),
        lr_scheduler=None,
        ignore_gpu_batch_transform_error=True,
        normalizers={},
        name='jmp',
        ckpt_path=Path('/mnt/shared/checkpoints/jmp-l.pt'),
        graph_computer=JMPGraphComputerConfig(
            pbc=True,
            cutoffs=CutoffsConfig(main=12.0, aeaint=12.0, qint=12.0, aint=12.0),
            max_neighbors=MaxNeighborsConfig(main=30, aeaint=20, qint=8, aint=1000),
            per_graph_radius_graph=False
        )
    ),
    trainer=TrainerConfig(
        accelerator='auto',
        strategy='auto',
        num_nodes=1,
        devices='auto',
        precision='32-true',
        deterministic=None,
        max_epochs=None,
        min_epochs=None,
        max_steps=-1,
        min_steps=None,
        max_time=None,
        val_check_interval=None,
        check_val_every_n_epoch=1,
        log_every_n_steps=None,
        gradient_clip_val=None,
        gradient_clip_algorithm=None,
        checkpoint=None,
        early_stopping=None,
        loggers='default',
        additional_trainer_kwargs={'fast_dev_run': True}
    )
)

In [ ]:
output = MatterTuner(hp).tune()
output

In [3]:
property_predictor = output.model.property_predictor()
print(property_predictor)

calculator = output.model.ase_calculator()
print(calculator)

In [4]:
import ase

# Create a test periodic system
atoms = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
print(atoms)

print(property_predictor.predict([atoms]))

You are running in `Trainer(barebones=True)` mode. All features that may impact raw speed have been disabled to facilitate analyzing the Trainer overhead. Specifically, the following features are deactivated:
 - Checkpointing: `Trainer(enable_checkpointing=True)`
 - Progress bar: `Trainer(enable_progress_bar=True)`
 - Model summary: `Trainer(enable_model_summary=True)`
 - Logging: `Trainer(logger=True)`, `Trainer(log_every_n_steps>0)`, `LightningModule.log(...)`, `LightningModule.log_dict(...)`
 - Sanity checking: `Trainer(num_sanity_val_steps>0)`
 - Development run: `Trainer(fast_dev_run=True)`
 - Anomaly detection: `Trainer(detect_anomaly=True)`
 - Profiling: `Trainer(profiler=...)`
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda)

Atoms(symbols='H2O', pbc=True, cell=[10.0, 10.0, 10.0])


/opt/conda/envs/jmp-peft/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


[{'energy': tensor[1] [-6.336], 'stresses': tensor[1, 3, 3] n=9 x∈[-0.253, 2.674e-06] μ=-0.093 σ=0.117 [[[-0.253, -6.240e-07, 2.674e-06], [-6.240e-07, -0.238, -0.047], [2.674e-06, -0.047, -0.250]]], 'forces': tensor[3, 3] n=9 x∈[-0.955, 1.512] μ=0.016 σ=0.839 [[-0.002, -0.955, -0.397], [0.000, 1.194, -0.852], [-0.002, -0.353, 1.512]]}]


In [5]:
import ase

# Create a test periodic system
atoms = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
print(atoms)

# Set the calculator
atoms.calc = calculator

# Calculate the energy
energy = atoms.get_potential_energy()
print(energy)

You are running in `Trainer(barebones=True)` mode. All features that may impact raw speed have been disabled to facilitate analyzing the Trainer overhead. Specifically, the following features are deactivated:
 - Checkpointing: `Trainer(enable_checkpointing=True)`
 - Progress bar: `Trainer(enable_progress_bar=True)`
 - Model summary: `Trainer(enable_model_summary=True)`
 - Logging: `Trainer(logger=True)`, `Trainer(log_every_n_steps>0)`, `LightningModule.log(...)`, `LightningModule.log_dict(...)`
 - Sanity checking: `Trainer(num_sanity_val_steps>0)`
 - Development run: `Trainer(fast_dev_run=True)`
 - Anomaly detection: `Trainer(detect_anomaly=True)`
 - Profiling: `Trainer(profiler=...)`
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda)

Atoms(symbols='H2O', pbc=True, cell=[10.0, 10.0, 10.0])


-6.336277961730957


In [6]:
property_predictor = output.model.property_predictor()
atoms_1 = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
atoms_2 = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
atoms = [atoms_1, atoms_2]
predictions = property_predictor.predict(atoms, ["energy", "forces"])
print("ase.Atoms 1 energy:", predictions[0]["energy"])
print("ase.Atoms 1 forces:", predictions[0]["forces"])
print("ase.Atoms 2 energy:", predictions[1]["energy"])
print("ase.Atoms 2 forces:", predictions[1]["forces"])

You are running in `Trainer(barebones=True)` mode. All features that may impact raw speed have been disabled to facilitate analyzing the Trainer overhead. Specifically, the following features are deactivated:
 - Checkpointing: `Trainer(enable_checkpointing=True)`
 - Progress bar: `Trainer(enable_progress_bar=True)`
 - Model summary: `Trainer(enable_model_summary=True)`
 - Logging: `Trainer(logger=True)`, `Trainer(log_every_n_steps>0)`, `LightningModule.log(...)`, `LightningModule.log_dict(...)`
 - Sanity checking: `Trainer(num_sanity_val_steps>0)`
 - Development run: `Trainer(fast_dev_run=True)`
 - Anomaly detection: `Trainer(detect_anomaly=True)`
 - Profiling: `Trainer(profiler=...)`
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda)

ase.Atoms 1 energy: tensor[1] [-6.336]
ase.Atoms 1 forces: tensor[3, 3] n=9 x∈[-0.955, 1.512] μ=0.016 σ=0.839 [[-0.002, -0.955, -0.397], [0.000, 1.194, -0.852], [-0.002, -0.353, 1.512]]
ase.Atoms 2 energy: tensor[1] [-6.336]
ase.Atoms 2 forces: tensor[3, 3] n=9 x∈[-0.955, 1.512] μ=0.016 σ=0.839 [[-0.002, -0.955, -0.397], [0.000, 1.194, -0.852], [-0.002, -0.353, 1.512]]
